In [17]:
import numpy as np, os

# --- physical constants ---
HARTREE_TO_J = 4.3597447222071e-18
BOHR_TO_M    = 5.29177210903e-11
AMU_TO_KG    = 1.66053906660e-27
C_CM_S       = 2.99792458e10
TWOPI        = 2*np.pi

def lam_to_cm1(lam):
    # lam in Hartree/(bohr^2·amu) -> cm^-1
    factor = HARTREE_TO_J / (BOHR_TO_M**2 * AMU_TO_KG)   # 1/s^2
    omega  = np.sqrt(lam * factor)                       # 1/s
    return omega / (TWOPI * C_CM_S)                      # cm^-1

# --- atomic masses in amu ---
MASS = {
    1: 1.00782503223,
    6: 12.0000000,
    7: 14.00307400443,
    8: 15.99491461957,
    9: 18.99840316273,
    17: 34.968852682,
}

def read_geom(path):
    """
    geom file format:
        3
        8.00000000000   x   y   z
        1.00000000000   x   y   z
        1.00000000000   x   y   z
    or optionally without the first '3' line.
    """
    lines = [ln.strip() for ln in open(path) if ln.strip()]
    first = lines[0].split()

    if len(first) == 1:
        start_idx = 1
    else:
        start_idx = 0
    Z_list = []
    coords_list = []
    for ln in lines[start_idx:]:
        parts = ln.split()
        if len(parts) < 4:
            raise ValueError(f"Bad geom line: {ln}")
        Z_list.append(int(float(parts[0])))
        coords_list.append([float(parts[1]), float(parts[2]), float(parts[3])])
    Z = np.array(Z_list, dtype=int)
    coords = np.array(coords_list, dtype=float)
    return Z, coords

def read_hess(path):
    """
    hessian file format:
        3
        h11 h12 h13
        h21 h22 h23
        ...
    It will:
      ① read ALL numbers
      ② first number is Nat
      ③ remaining numbers reshape to (3N,3N)
    """
    nums = []
    with open(path, "r") as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            for tok in ln.split():
                nums.append(float(tok))
    if len(nums) < 1:
        raise ValueError("Empty Hessian file")
    Nat = int(nums[0])
    data = np.array(nums[1:], dtype=float)
    dim = 3 * Nat
    expected = dim * dim
    if data.size != expected:
        raise ValueError(
            f"Hessian size mismatch: expected {expected} numbers for {dim}x{dim}, got {data.size}"
        )
    H = data.reshape((dim, dim))
    return Nat, H

def mass_weight(Z, H):
    """
    mass-weighted Hessian:
    H_M[i,j] = H[i,j] / sqrt(m_i m_j)
    where m_i = mass of atom for that Cartesian DOF
    """
    masses = np.array([MASS[z] for z in Z], float)   # amu per atom
    mvec = np.repeat(masses, 3)                      # length 3N
    denom = np.sqrt(np.outer(mvec, mvec))
    return H / denom

def run(geom_path, hess_path):
    Z, coords = read_geom(geom_path)
    Nat_hess, H = read_hess(hess_path)

    if len(Z) != Nat_hess:
        raise ValueError(
            f"Atom count mismatch: geometry has {len(Z)}, Hessian says {Nat_hess}"
        )

    HM = mass_weight(Z, H)
    eigvals, eigvecs = np.linalg.eigh(HM)

    # convert eigenvalues to frequencies
    freqs_cm1 = []
    for lam in eigvals:
        if lam <= 0:
            # imaginary -> mark negative
            fcm = -lam_to_cm1(abs(lam))
        else:
            fcm = lam_to_cm1(lam)
        freqs_cm1.append(fcm)

    print("\n=== Harmonic Vibrational Analysis ===")
    print(f"Atoms: {len(Z)}")
    print(f"DOF: {3*len(Z)}")
    print("--------------------------------------")
    print("Eigenvalues [Hartree/(bohr^2·amu)]:")
    for i, val in enumerate(eigvals, 1):
        print(f"{i:2d}: {val: .6e}")
    print("--------------------------------------")
    print("Frequencies (cm^-1):")
    print("negative value => imaginary (i)")
    for i, fr in enumerate(freqs_cm1, 1):
        if fr < 0:
            print(f"{i:2d}: {abs(fr):.2f} i cm^-1")
        else:
            print(f"{i:2d}: {fr:.2f} cm^-1")
    print("======================================\n")

if __name__ == "__main__":
    gpath = input("Geometry file path: ").strip('" ').strip()
    hpath = input("Hessian  file path: ").strip('" ').strip()
    gpath = os.path.expanduser(gpath)
    hpath = os.path.expanduser(hpath)
    run(gpath, hpath)


Geometry file path:  D:/h2o_geom.txt
Hessian  file path:  D:/h2o_hessian.txt



=== Harmonic Vibrational Analysis ===
Atoms: 3
DOF: 9
--------------------------------------
Eigenvalues [Hartree/(bohr^2·amu)]:
 1: -7.310268e-18
 2: -4.959881e-19
 3:  1.665694e-11
 4:  5.182166e-02
 5:  5.475515e-02
 6:  5.611240e-02
 7:  1.317513e-01
 8:  2.107113e-01
 9:  2.351542e-01
--------------------------------------
Frequencies (cm^-1):
negative value => imaginary (i)
 1: 0.00 i cm^-1
 2: 0.00 i cm^-1
 3: 0.02 cm^-1
 4: 1170.20 cm^-1
 5: 1202.86 cm^-1
 6: 1217.68 cm^-1
 7: 1865.87 cm^-1
 8: 2359.65 cm^-1
 9: 2492.76 cm^-1

